In [1]:
from class_function import Skipgram, SkipgramNeg, Glove

In [2]:
import torch
import torch.nn.functional as F
import pickle
import numpy as np
from scipy.stats import spearmanr

### Trained data

In [3]:
# Importing training data
Data = pickle.load(open(r'D:\AIT_lecture\NLP\code\Assignment\NLP-2025\NLP-A1\models\Data.pkl', 'rb'))

corpus = Data['corpus']
vocab = Data['vocab']
word2index = Data['word2index']
voc_size = Data['voc_size']
embed_size = Data['embedding_size']

### Models

In [4]:
import torch

# Load the state_dict
state_dict = torch.load(r'D:\AIT_lecture\NLP\code\Assignment\NLP-2025\NLP-A1\models\skipgram_v1.pt')

# Initialize the Skipgram model with the appropriate vocab_size and embed_size
skipgram = Skipgram(voc_size, embed_size)

# Load the state_dict into the model
skipgram.load_state_dict(state_dict,strict=False)

# Set the model to evaluation mode
skipgram.eval()


C:\Users\User\AppData\Local\Temp\ipykernel_4304\217274521.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(r'D:\AIT_lecture\NLP\code\Assignment\NL

Skipgram(
  (embedding_center): Embedding(14270, 2)
  (embedding_outside): Embedding(14270, 2)
)

In [5]:
import torch

# Load the state_dict
state_dict = torch.load(r'D:\AIT_lecture\NLP\code\Assignment\NLP-2025\NLP-A1\models\skipgramNeg_v1.pt')

# Remap keys
state_dict_remapped = {
    'embedding_center.weight': state_dict['center_embedding.weight'],
    'embedding_outside.weight': state_dict['outside_embedding.weight'],
}

# Load the remapped state_dict into the model
skipgramNeg = SkipgramNeg(voc_size, embed_size)
skipgramNeg.load_state_dict(state_dict_remapped)
skipgramNeg.eval()




C:\Users\User\AppData\Local\Temp\ipykernel_4304\1708150165.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(r'D:\AIT_lecture\NLP\code\Assignment\N

SkipgramNeg(
  (embedding_center): Embedding(14270, 2)
  (embedding_outside): Embedding(14270, 2)
  (logsigmoid): LogSigmoid()
)

In [7]:
 #Initialize the model and load the pre-trained weights from the saved file
glove = Glove(voc_size, embed_size)
glove.load_state_dict(torch.load(r'D:\AIT_lecture\NLP\code\Assignment\NLP-2025\NLP-A1\models\GloVe-v1.pt'))
glove.eval()

C:\Users\User\AppData\Local\Temp\ipykernel_4304\1039450169.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  glove.load_state_dict(torch.load(r'D:\AIT_lecture\NLP\code\Ass

Glove(
  (center_embedding): Embedding(14270, 2)
  (outside_embedding): Embedding(14270, 2)
  (center_bias): Embedding(14270, 1)
  (outside_bias): Embedding(14270, 1)
)

In [8]:
import torch
import pickle

In [9]:
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [10]:
#search glove.6G.100d.txt from google, and save 
glove_file = datapath('glove.6B.100d.txt')  
gensim = KeyedVectors.load_word2vec_format(glove_file, binary=False, no_header=True)


### Similarity Function

In [11]:
import torch
import torch.nn.functional as F

def compute_all_word_vectors(vocab, model):
    """
    Compute vectors for all words in the vocabulary.
    
    Args:
        vocab (list): List of vocabulary words.
        model (object): Pretrained embedding model.
    
    Returns:
        torch.Tensor: A tensor containing vectors for all vocabulary words.
    """
    word_vectors = []
    for word in vocab:
        word_vectors.append(model.get_vector(word))
    return torch.stack(word_vectors)

def similarities(lines, model, vocab):
    """
    Perform semantic and syntactic analysis using word vector similarities.
    
    Args:
        lines (list): List of analogies (each line has four words: a, b, c, d).
        model (object): Pretrained embedding model.
        vocab (list): List of vocabulary words.
    
    Returns:
        float: Accuracy of the analogies.
    """
    # Compute all word vectors once
    all_word_vectors = compute_all_word_vectors(vocab, model)

    correct = 0
    skipped = 0

    for line in lines:
        words = line.split()

        # Skip lines with unknown words
        if any(word not in vocab for word in words):
            skipped += 1
            # print(f"Skipping analogy due to unknown words: {line}")
            continue

        # Retrieve vectors for analogy words
        vectors = [model.get_vector(word.lower()) for word in words]

        # Perform vector manipulation
        result_vector = vectors[1] - vectors[0] + vectors[2]
        result_vector = result_vector.unsqueeze(0)  # Add batch dimension

        # Calculate cosine similarities
        similarities = F.cosine_similarity(result_vector, all_word_vectors)

        # Get the closest word
        closest_word_index = torch.argmax(similarities).item()
        closest_word = vocab[closest_word_index]

        if closest_word == words[3]:  # Check if predicted word matches target
            correct += 1
        # else:
        #     print(f"Mismatch: {line} -> Predicted: {closest_word}")

    # Calculate accuracy
    total = len(lines) - skipped
    accuracy = (correct / total) * 100 if total > 0 else 0
    print('---------------------------------------------------------')
    print(f'Accuracy: {accuracy:.2f}%')
    print(f'Skipped: {skipped} analogies (unknown words)')
    return accuracy


# 1. Semantic and Syntatic Analysis

## Load test text files

In [12]:
# Read the text file and create a list of tuples
with open(r'D:\AIT_lecture\NLP\code\Assignment\NLP-2025\NLP-A1\dataset\semantic_capital_country.txt', 'r') as file:
    sem_lines = file.readlines()

In [13]:
# Read the text file and create a list of tuples
with open(r'D:\AIT_lecture\NLP\code\Assignment\NLP-2025\NLP-A1\dataset\syntatic_past_tense.txt', 'r') as file:
    syn_lines = file.readlines()

## Work2Vec(Skipgram)

### Semantic

In [14]:
similarities(sem_lines, skipgram,vocab)

---------------------------------------------------------
Accuracy: 0.00%
Skipped: 506 analogies (unknown words)


0

### Syntatic

In [15]:
similarities(syn_lines, skipgram,vocab)

---------------------------------------------------------
Accuracy: 0.00%
Skipped: 1008 analogies (unknown words)


0.0

## Skipgram Negative Sampling model

### Semantic

In [16]:
similarities(sem_lines, skipgramNeg,vocab)

---------------------------------------------------------
Accuracy: 0.00%
Skipped: 506 analogies (unknown words)


0

### Syntatic

In [17]:
similarities(syn_lines, skipgramNeg,vocab)

---------------------------------------------------------
Accuracy: 0.00%
Skipped: 1008 analogies (unknown words)


0.0

## GloVe Model

### Semantic

In [18]:
similarities(sem_lines, glove,vocab)

---------------------------------------------------------
Accuracy: 0.00%
Skipped: 506 analogies (unknown words)


0

### Syntatic

In [19]:
similarities(syn_lines, glove,vocab)

---------------------------------------------------------
Accuracy: 0.00%
Skipped: 1008 analogies (unknown words)


0.0

## GloVe(Gensim) Model

In [20]:
def evaluate_glove(lines, model):
    """
    Evaluate GloVe (Gensim) model on word analogy tasks.

    Args:
        lines (list): List of analogy lines (e.g., "king queen man woman").
        model: Gensim KeyedVectors model (e.g., GloVe).

    Returns:
        float: Accuracy of the analogy task.
    """
    correct = 0
    total = 0

    for line in lines:
        words = line.lower().split()  # Convert line to lowercase and split into words

        # Check if line is valid and all words exist in the model
        if len(words) != 4:
            print(f"Skipping malformed line: {line}")
            continue
        if any(word not in model for word in words):
            print(f"Skipping line due to OOV words: {line}")
            continue

        # Perform analogy
        try:
            result = model.most_similar(positive=[words[2], words[1]], negative=[words[0]], topn=1)
            closest_word = result[0][0]  # Get the most similar word
            total += 1

            if closest_word == words[3]:
                correct += 1
        except Exception as e:
            print(f"Error processing line {line}: {e}")
            continue

    accuracy = (correct / total) * 100 if total > 0 else 0
    print('---------------------------------------------------------')
    print(f'Total lines evaluated: {total}')
    print(f'Accuracy: {accuracy:.2f}%')
    return accuracy


### Semantic

In [21]:
evaluate_glove(sem_lines,gensim)

---------------------------------------------------------
Total lines evaluated: 506
Accuracy: 93.87%


93.87351778656127

### Syntatic

In [22]:
evaluate_glove(syn_lines,gensim)

---------------------------------------------------------
Total lines evaluated: 1560
Accuracy: 55.45%


55.44871794871795

# 2. Similarity Analysis

In [23]:
# Read the text file and create a list of tuples
with open(r'D:\AIT_lecture\NLP\code\Assignment\NLP-2025\NLP-A1\dataset\wordsim_similarity_goldstandard.txt', 'r') as file:
    similarity_lines = file.readlines()

The cosine similarity between two vectors \( A \) and \( B \) is calculated using the formula:

$$
\text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}
$$


In [24]:
import numpy as np

def cosine_similarity(A, B):
    """
    Compute cosine similarity between two vectors A and B.

    Args:
        A (numpy array): First vector.
        B (numpy array): Second vector.

    Returns:
        float: Cosine similarity between A and B.
    """
    # Flatten vectors to ensure they're 1D
    A = A.flatten()
    B = B.flatten()

    # Compute dot product and norms
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)

    # Return cosine similarity
    similarity = dot_product / (norm_a * norm_b)
    return similarity


In [25]:
from scipy.stats import spearmanr

def similar(lines, model):
    """
    Evaluate word similarity using cosine similarity and Spearman rank correlation.

    Args:
        lines (list of str): List of sentences with 3 parts: word1, word2, real_similarity_score.
        model (gensim model): Pretrained word embedding model (e.g., GloVe, Word2Vec).

    Returns:
        spearmanr object: Spearman rank correlation of predicted vs actual similarity scores.
    """
    scores_real = []  # To store actual similarity scores (from the dataset)
    scores_pred = []  # To store predicted similarity scores (using cosine similarity)

    for line in lines:
        words = line.split()  # Split line into words
        vec = []  # List to store word vectors

        # Assuming the first two words need to be compared
        for word in words[:2]: 
            try:
                # Attempt to get the vector for the word
                vec.append(model.get_vector(word).detach().numpy())
            except:
                # If the word is not in the vocabulary, use the <UNK> token
                vec.append(model.get_vector('<UNK>').detach().numpy())

        # Store the actual similarity score from the dataset (third word)
        scores_real.append(float(words[2]))  
        
        # Calculate the cosine similarity between the two words and store the predicted score
        scores_pred.append(cosine_similarity(np.array(vec[0]), np.array(vec[1])))

    # Calculate and return Spearman's rank correlation between actual and predicted scores
    return spearmanr(scores_real, scores_pred)



## Skipgram model

In [26]:
print(f'Skipgram correlation score: {similar(similarity_lines,skipgram)[0]}')

Skipgram correlation score: 0.19211968837499357


## Skipgram (Negative sampling) model

In [27]:
print(f'Skipgram (Neg) correlation score: {similar(similarity_lines,skipgramNeg)[0]}')

Skipgram (Neg) correlation score: 0.12842785877674645


## GloVe model

In [28]:
print(f'GLove correlation score: {similar(similarity_lines,glove)[0]}')

GLove correlation score: 0.25997528190281816


## GloVe (Gensim) model

In [29]:
def similar_gensim(lines, model):
    scores_real = []  # Store real human similarity scores
    scores_pred = []  # Store predicted cosine similarities based on embeddings

    for line in lines:
        words = line.split()  # Split each line into words
        vec = []
        
        # Extract word vectors for the first two words
        for word in words[:2]:
            try:
                # Use model[word] to get the embedding directly from the model (Gensim)
                vec.append(model[word])
            except KeyError:
                # Handle missing words by using a placeholder or a default embedding
                # Here I use 'unk' as an example. Adjust based on your vocabulary.
                vec.append(model['unk'])  # You can use your own word for unknown words.
        
        # Append human similarity score (the third element of each line)
        scores_real.append(float(words[2]))

        # Compute the predicted similarity using cosine similarity
        similarity_score = cosine_similarity(np.array(vec[0]), np.array(vec[1]))
        scores_pred.append(similarity_score)

    # Calculate Spearman's rank correlation between real and predicted scores
    correlation, p_value = spearmanr(scores_real, scores_pred)

    # print(f"Spearman Rank Correlation of Gensim: {correlation:.4f}")
    # print(f"P-value: {p_value:.4f}")
    
    return correlation, p_value

In [30]:
print(f'Gensim correlation score: {similar_gensim(similarity_lines,gensim)[0]}')

Gensim correlation score: 0.6035596038474791


## Human Judgment

In [31]:
from scipy.stats import spearmanr

def similar_human(lines):
    scores_real = []  # Store human-provided similarity scores
    scores_pred = []  # Store predicted scores (from the user)

    for line in lines:
        words = line.split()
        scores_real.append(float(words[2]))

        while True:
            try:
                # Ask the user for input
                human_score = float(input(f"How would you rate the relation between '{words[0]}' and '{words[1]}' on a scale of 0 to 10: "))
                
                # Check if the input is within the valid range (0 to 10)
                if 0 <= human_score <= 10:
                    scores_pred.append(human_score)
                    break  # Exit the loop if the input is valid
                else:
                    print("Invalid input. Please enter a score between 0 and 10.")

            except ValueError:
                print("Invalid input. Please enter a numeric value.")

    # Compute Spearman's Rank Correlation
    correlation, p_value = spearmanr(scores_real, scores_pred)

    return correlation, p_value





In [32]:
correlation, p_value = similar_human(similarity_lines)
print(f"Human correlation score: {correlation:.4f}")


Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Invalid input. Please enter a numeric value.
Human correlation score: 0.8735


# Test score comparison are provided in README.md